In [ ]:
import os, sys

sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
from nbafuns import *

player_dict = get_players_pbp()
teams_dict, teams_list = get_teams()

box_DIR = "../data/box/"
img_DIR_P = "../data/images/players/"
fig_DIR = "../figs/players/"

In [ ]:
seasons = np.arange(2024,2025)
df1 = get_box("P","Base",False,seasons)
df1["game_id"] = df1["game_id"].astype(int)
df2 = get_box("P","Adv",False,seasons)
df2 = df2[["gameid","personid","possessions"]]
df2.columns = ['game_id','player_id',"poss"]
df3 = pd.merge(df1,df2,on=['game_id','player_id'])
df3["dre"] = -8.42364*df3["poss"]/100+0.79231*df3["pts"]-0.71944*(df3["fga"]-df3["fg3a"])-0.55233*df3["fg3a"]-0.15944*df3["fta"]+0.13479*df3["oreb"]+0.3996*df3["dreb"]+0.54415*df3["ast"]+1.68007*df3["stl"]+0.76387*df3["blk"]-1.3599*df3["tov"]-0.10838*df3["pf"]

df3["gmsc"] = df3["pts"]+0.4*df3["fgm"]-0.7*df3["fga"]-0.4*(df3["fta"]-df3["ftm"])+0.7*df3["oreb"]+0.3*df3["dreb"]+df3["stl"]+0.7*df3["ast"]+0.7*df3["blk"]-0.4*df3["pf"]-df3["tov"]

In [ ]:
df4 = df3[['player_id', 'player_name', 'game_date', 'dre', 'season']]
df4 = df4.sort_values(["game_date","player_id"])
df4g = df4.groupby(['player_id', 'player_name', 'season'])
keys = list(df4g.groups)

In [ ]:
window = 7
dfa = []
for key in keys:
    df5 = df4g.get_group(key)
    df5["rdre"] = df5["dre"].rolling(window=window).mean()
    df5 = df5.dropna()
    df6 = df5.head(1)
    df6["cold"] = df5["rdre"].min()
    df6["hot"] = df5["rdre"].max()
    df6["diff"] = df6["hot"]-df6["cold"]
    df6["gp"] = df5["rdre"].count()
    dfa.append(df6)
df7 = pd.concat(dfa)
df7 = df7[['player_id', 'player_name', 'season', 'gp', 'hot', 'cold', 'diff']]
df7.iloc[:,3:] = df7.iloc[:,3:].round(1)
df7['player_name'] = (
    df7['player_name']
    .str.normalize('NFKD')
    .str.encode('ascii', errors='ignore')
    .str.decode('utf-8')
) 

In [ ]:
df71 = get_box("P","Base",True,seasons)
df71 = df71[['player_id','team_abbreviation']]
df72 = pd.merge(df71,df7)
df72.insert(2,"team_abbreviation",df72.pop("team_abbreviation"))
df72.pop("season")
df72.head(5)

In [ ]:
df8 = df72.sort_values("diff",ascending=False).reset_index(drop=True)
df9 = df72.sort_values("diff",ascending=True).reset_index(drop=True)

In [ ]:
df10 = df8.query("cold <= -2").reset_index(drop=True).reset_index()
df10["index"] +=1
data = df10.copy()
data["player_id"] = data["player_id"].astype(str)
n = 10
data = data.head(n)
# len(data1)

In [ ]:

title = f"**Largest difference in Hottest and Coldest {window} game stretches**"
subtitle = '''
Evaluation metric is **DRE** (Daily RAPM Estimate) by Kevin Ferrigan  
Only Cold streches with DRE<=-2 are considered
'''
t = (
    GT(data)
    .tab_header(
        title=md(title),
        subtitle=md(subtitle)
    )
    .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker")
    .cols_label(
        index = "#",
        player_id = "",
        team_abbreviation="Team",
        player_name="Player",
        hot="Hot",
        cold="Cold",
        diff="Difference",
    )
    .fmt_image(
        columns="player_id",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="player_name")
    .data_color(
        domain = [-5,10],
        columns=['hot','cold'],
        palette="RdBu",
        reverse="False"
    )
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="ghostwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        # table_margin_left = 7,
        # table_margin_right = 0,
    )
)
t.save(fig_DIR + f"hot_cold_diff_"+ str(seasons[-1]) +".png",scale=3,web_driver="firefox")
t

In [ ]:
team = "IND"
df10 = df8.query(f"team_abbreviation == '{team}'").reset_index(drop=True).reset_index()
df10["index"] +=1
data = df10.copy()
data["player_id"] = data["player_id"].astype(str)

In [ ]:

title = f"**{team}: Largest difference in Hottest and Coldest {window} game stretches**"
subtitle = '''
Evaluation metric is **DRE** (Daily RAPM Estimate) by Kevin Ferrigan  
Only Cold streches with DRE<=-2 are considered
'''
t = (
    GT(data)
    .tab_header(
        title=md(title),
        subtitle=md(subtitle)
    )
    .tab_source_note(source_note="bsky:@sradjoker.cc | X:@sradjoker")
    .cols_label(
        index = "#",
        player_id = "",
        team_abbreviation="Team",
        player_name="Player",
        hot="Hot",
        cold="Cold",
        diff="Difference",
    )
    .fmt_image(
        columns="player_id",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="player_name")
    .data_color(
        domain = [-5,10],
        columns=['hot','cold'],
        palette="RdBu",
        reverse="False"
    )
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="ghostwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        # table_margin_left = 7,
        # table_margin_right = 0,
    )
)
# t.save(fig_DIR + f"hot_cold_diff_"+ str(seasons[-1]) +".png",scale=3,web_driver="firefox")
# t.save(fig_DIR + f"hot_cold_diff_{team}_"+ str(seasons[-1]) +".png",scale=3,web_driver="firefox")
t

In [ ]:
df11 = df9.query("hot >= 5 & gp>=25").reset_index(drop=True).reset_index()
df11["index"] +=1
data = df11.copy()
data["player_id"] = data["player_id"].astype(str)
len(data)

In [ ]:
n = 10
title = "**Smallest difference in Hottest and Coldest 7 game stretches**"
subtitle = '''
Evaluation metric is **DRE** (Daily RAPM Estimate) by Kevin Ferrigan  
Only Cold streches with DRE>=2 are considered | Min 25 games played
'''
t = (
    GT(data.head(n))
    .tab_header(
        title=md(title),
        subtitle=md(subtitle)
    )
    .tab_source_note(source_note="bsky:@sradjoker.cc | x:@sradjoker")
    .cols_label(
        index = "#",
        player_id = "",
        player_name="Player",
        hot="Hot",
        cold="Cold",
        diff="Difference",
    )
    .fmt_image(
        columns="player_id",
        path = img_DIR_P,
        file_pattern="{}.png"
    )
    .cols_align(align="center")
    .cols_align(align="left", columns="player_name")
    .data_color(
        domain = [0,11],
        columns=['hot','cold'],
        palette="Reds",
        reverse=False
    )
    .tab_options(
        heading_title_font_size="150%",
        heading_subtitle_font_size="110%",
        # heading_title_font_weight='bold',
        table_background_color="ghostwhite",
        column_labels_font_size="105%",
        column_labels_font_weight='bold',
        row_group_font_weight='bold',
        row_group_background_color="#E5E1D8",
        table_font_size=10,
        table_font_names="Consolas", 
        data_row_padding = "3px",
        # table_margin_left = 7,
        # table_margin_right = 0,
    )
)
# t.save(fig_DIR + f"hot_cold_diff_"+ str(season) +".png",scale=3,web_driver="firefox")
t